In [7]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

from operator import itemgetter

## Generate a Doubly Stochastic Matrix

In [8]:
def gen_P(n):
    P = np.random.random((n, n))
    
    rsum = 0
    csum = 0

    while (np.any(np.round(rsum, 3) != 1)) | (np.any(np.round(csum, 3) != 1)):
        P /= P.sum(0)
        P = P / P.sum(1)[:, np.newaxis]
        rsum = P.sum(1)
        csum = P.sum(0)
        
    return P

n = 3
P = gen_P(n)
print(P)

[[0.26213016 0.40661542 0.33125442]
 [0.31404933 0.16766826 0.51828241]
 [0.42385335 0.42581003 0.15033662]]


## Find "Closest" permutation matrix
Using the Hungarian Algorithm, we can find the "closest" permutation matrix $P_{perm}$ to $P_{DS}$.

In [16]:
from scipy import optimize

def closest_perm_1(P_DS):
    # convert to get the maximum linear assignment problem
    P_MOD = np.ones((n, n)) * np.max(P_DS) - P_DS
    row_ind, col_ind = optimize.linear_sum_assignment(P_MOD)

    # Create Permutation matrix to return
    P_perm = np.zeros((n, n))

    for row, col in zip(row_ind, col_ind):
        P_perm[row][col] = 1 
    
    return P_perm

P_DS = gen_P(n)
print("Doubly stochastic P:\n", P_DS, end = "\n\n")
print("Hungarian Algorithm:\n", closest_perm_1(P_DS), end = "\n\n")
print("Maximum to one:\n", closest_perm_2(P_DS), end = "\n\n")
print("Relative maximum in row to one:\n", closest_perm_3(P_DS), end = "\n\n")
print("Relative maximum in column to one:\n", closest_perm_4(P_DS), end = "\n\n")

Doubly stochastic P:
 [[0.45412022 0.44072023 0.10515955]
 [0.39080582 0.273181   0.33601318]
 [0.15499343 0.28605872 0.55894786]]

Hungarian Algorithm:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

Maximum to one:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

Relative maximum in row to one:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Relative maximum in column to one:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]



In [2]:
def closest_perm_2(P_DS):
    P_2 = P_DS.copy()
    # for each row
    for _ in range(n):
        # Normalize each row by dividing by its smallest non-zero entry
        for j in range(n):
            P_2[:, j] /= np.min(P_2[:, j][np.nonzero(P_2[:, j])])

        # get largest entry in P
        i, j = np.unravel_index(np.argmax(P_2, axis=None), P_2.shape)

        # set all other values in that row and column to zero
        for i1 in range(n):
            P_2[i][i1] = 0
            P_2[i1][j] = 0

        # set the largest entry in P to 1
        P_2[i][j] = 1

    return P_2

In [3]:
def closest_perm_3(P_DS):
    P_2 = P_DS.copy()
    # for each row
    for _ in range(n):
        # Normalize each row by dividing by its smallest non-zero entry
        for i in range(n):
            P_2[i] /= np.min(P_2[i][np.nonzero(P_2[i])])


        # get largest entry in P
        i, j = np.unravel_index(np.argmax(P_2, axis=None), P_2.shape)

        # set all other values in that row and column to zero
        for i1 in range(n):
            P_2[i][i1] = 0
            P_2[i1][j] = 0

        # set the largest entry in P to 1
        P_2[i][j] = 1

    return P_2

In [4]:
def closest_perm_4(P_DS):
    P_2 = P_DS.copy() * -1
    # for each row
    for _ in range(n):
        # get largest entry in P
        i, j = np.unravel_index(np.argmin(P_2, axis=None), P_2.shape)
    
        # set all other values in that row and column to zero
        for i1 in range(n):
            P_2[i][i1] = 0
            P_2[i1][j] = 0
        
        # set the largest entry in P to 1
        P_2[i][j] = 1

    return P_2